In [1]:
%load_ext autoreload
%autoreload 2
import pprint, json, math, os, sys
# dir_path = "C:\\Users\\rando\\OneDrive\\Documents\\mywork-repo"
# fund_path = r"C:\Users\rando\OneDrive\Documents\PDFDrive\Mar25"
# dry_path = r"C:\Users\rando\OneDrive\Documents\mywork-repo\notebook\DryRun.pdf"

dir_path = "C:\\Users\\Kaustubh.keny\\Projects\\office-work\\mywork-repo"
fund_path = r"C:\Users\Kaustubh.keny\Projects\PDF\Apr 25"
dry_path = r"C:\Users\Kaustubh.keny\Projects\office-work\mywork-repo\notebook\DryRun.pdf"
sys.path.append(os.path.abspath(dir_path))
from app.config_loader import load_config_once
conf = load_config_once()


import fitz, pdfplumber, ocrmypdf,camelot, pprint
import pandas as pd
import numpy as np
from collections import defaultdict

from app.utils import Helper
from app.parse_regex import *
from app.parse_table import *

dry_path = r'DryRun.pdf'
fin_path = r'\data\input\financial_indices.xlsx'
mutual_fund = Helper.get_fund_paths(fund_path)

Loaded config with output_folder = None


In [69]:
mutual_fund["Axis Mutual Fund"]

'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\Apr 25\\Axis Mutual Fund\\1_30-Apr-2025_1_FS.pdf'

In [ ]:
lines = [
    ((110, 0), (110, 812)),# Vertical line
    ((0, 350), (812, 350)),
    ((570, 0), (570, 812))
]
pages = [12, 14,16]
bboxes = [[0, 120, 180, 812],[180, 85, 360, 812]] #[(0, 85, 180, 812),(180, 85, 360, 812),(0,100,270,812),(0,100,350,812)]
pages = [i for i in range(1,110)]
Helper.draw_lines_on_pdf(sample_path, lines, bboxes, pages, dry_path)

In [17]:
axis_apr = r"C:\Users\Kaustubh.keny\Projects\PDF\Apr 25\Axis Mutual Fund\1_30-Apr-25_FS.pdf"
table_parser = TableParser()
tables = camelot.read_pdf(axis_apr,flavor="hybrid",pages="122-123")
dfs = pd.concat([table.df for table in tables], ignore_index=True)


In [ ]:
dfs.columns = ["MUTUAL_FUND","MIN_APP","ADD_APP"]+ [f"OTHER_{i}" for i in range(1,(dfs.shape[1]-3)+1)]
axis = re.compile(
    "axis.+",
    re.IGNORECASE
)
fdf= dfs
dfs.MUTUAL_FUND = table_parser.clean_series(dfs.MUTUAL_FUND,["normalize_alphanumeric"])
fdf.MUTUAL_FUND = dfs.MUTUAL_FUND.apply(lambda x: axis.findall(x)[0] if isinstance(x, str) and axis.findall(x) else "")
fdf = table_parser.clean_dataframe(dfs,["newline_to_space","str_to_pd_NA"])
fdf = fdf.dropna(axis=0, how="all").dropna(axis=1, how="all")
fdf =table_parser.clean_dataframe(fdf,['NA_to_str'])



In [ ]:
fdf.head(10)

In [51]:
final_dict = {}
current = None

for _, row in fdf.iterrows():
    first, second, third, *rest = row

    if isinstance(first, str) and first.lower().startswith("axis"):
        current = first
        final_dict[current] = {
            "min_amt": [second],
            "add_amt": [third]
        }
    elif current:
        final_dict[current]["min_amt"].append(second)
        final_dict[current]["add_amt"].append(third)


In [44]:
final_dict  = {}
current = "dummy"
final_dict[current] = {"min_amt":[],"add_amt":[]}
for idx, rows in fdf.iterrows():
    first,second,third,*fourth = rows
    if first and first.startswith("axis"):
        if first not in final_dict:
            final_dict[first] ={"min_amt":[second],"add_amt":[third]}
            current = first
    
    if not first:
        temp = final_dict[current]
        temp["min_amt"].append(second)
        temp["add_amt"].append(third)
        final_dict[first] = temp



In [71]:
axis_regex = {
            "Axis Large Cap Fund": "(Axis\\s+Large\\s+Cap\\s+Fund|Axis\\s*Bluechip\\s*fund)",
            "Axis Focused Fund": "Axis\\s+Focused\\s+Fund",
            "Axis ELSS Tax Saver Fund": "Axis\\s+ELSS\\s+Tax\\s+Saver\\s+Fund",
            "Axis Large & Mid Cap Fund": "(Axis\\s+Large\\s*(?:&|and)?\\s*Mid\\s*Cap\\s*Fund|Axis\\s*Growth\\s*Opportunities\\s*Fund)",
            "Axis Flexi Cap Fund": "Axis\\s+Flexi\\s+Cap\\s+Fund",
            "Axis Multicap Fund": "Axis\\s+Multi\\s*cap\\s+Fund",
            "Axis Midcap Fund": "Axis\\s+Mid\\s*cap\\s+Fund",
            "Axis Innovation Fund": "Axis\\s+Innovation\\s+Fund",
            "Axis Small Cap Fund": "Axis\\s+Small\\s+Cap\\s+Fund",
            "Axis ESG Integration Strategy Fund": "Axis\\s+ESG\\s+Integration\\s+Strategy\\s+Fund",
            "Axis Quant Fund": "Axis\\s+Quant\\s+Fund",
            "Axis Value Fund": "Axis\\s+Value\\s+Fund",
            "Axis Business Cycles Fund": "Axis\\s+Business\\s+Cycles\\s+Fund",
            "Axis India Manufacturing Fund": "Axis\\s+India\\s+Manufacturing\\s+Fund",
            "Axis Consumption Fund": "Axis\\s+Consumption\\s+Fund",
            "Axis Momentum Fund": "Axis\\s+Momentum\\s+Fund",
            "Axis Nifty 50 ETF": "Axis\\s+Nifty\\s+50\\s+ETF",
            "Axis Nifty Bank ETF": "Axis\\s+Nifty\\s+Bank\\s+ETF",
            "Axis Nifty IT ETF": "Axis\\s+Nifty\\s+IT\\s+ETF",
            "Axis Nifty Healthcare ETF": "Axis\\s+Nifty\\s+Healthcare\\s+ETF",
            "Axis Nifty India Consumption ETF": "Axis\\s+Nifty\\s+India\\s+Consumption\\s+ETF",
            "Axis BSE Sensex ETF": "Axis\\s+BSE\\s+Sensex\\s+ETF",
            "Axis Nifty500 Value 50 ETF": "Axis\\s+Nifty500\\s+Value\\s+50\\s+ETF",
            "Axis Nifty 100 Index Fund": "Axis\\s+Nifty\\s+100\\s+Index\\s+Fund",
            "Axis Nifty 50 Index Fund": "Axis\\s+Nifty\\s+50\\s+Index\\s+Fund",
            "Axis BSE Sensex Index Fund": "Axis\\s+BSE\\s+Sensex\\s+Index\\s+Fund",
            "Axis Nifty Next 50 Index Fund": "Axis\\s+Nifty\\s+Next\\s+50\\s+Index\\s+Fund",
            "Axis Nifty Smallcap 50 Index Fund": "Axis\\s+Nifty\\s+Smallcap\\s+50\\s+Index\\s+Fund",
            "Axis Nifty Midcap 50 Index Fund": "Axis\\s+Nifty\\s+Midcap\\s+50\\s+Index\\s+Fund",
            "Axis Nifty IT Index Fund": "Axis\\s+Nifty\\s+IT\\s+Index\\s+Fund",
            "Axis Nifty Bank Index Fund": "Axis\\s+Nifty\\s+Bank\\s+Index\\s+Fund",
            "Axis Nifty 500 Index Fund": "Axis\\s+Nifty\\s+500\\s+Index\\s+Fund",
            "Axis Nifty500 Value 50 Index Fund": "Axis\\s+Nifty500\\s+Value\\s+50\\s+Index\\s+Fund",
            "Axis Nifty500 Momentum 50 Index Fund": "Axis\\s+Nifty500\\s+Momentum\\s+50\\s+Index\\s+Fund",
            "Axis Equity ETFs FOF": "Axis\\s+Equity\\s+ETFs\\s+FOF",
            "Axis Global Equity Alpha Fund of Fund": "Axis\\s+Global\\s+Equity\\s+Alpha\\s+Fund\\s+of\\s+Fund",
            "Axis Greater China Equity Fund of Fund": "Axis\\s+Greater\\s+China\\s+Equity\\s+Fund\\s+of\\s+Fund",
            "Axis Global Innovation Fund of Fund": "Axis\\s+Global\\s+Innovation\\s+Fund\\s+of\\s+Fund",
            "Axis Nasdaq 100 Fund of Fund": "Axis\\s+Nasdaq\\s+100\\s+Fund\\s+of\\s+Fund",
            "Axis Overnight Fund": "Axis\\s+Overnight\\s+Fund",
            "Axis Liquid Fund": "Axis\\s+Liquid\\s+Fund",
            "Axis Ultra Short Duration Fund": "Axis\\s+Ultra\\s+Short\\s+Duration\\s+Fund",
            "Axis Floater Fund": "Axis\\s+Floater\\s+Fund",
            "Axis Treasury Advantage Fund": "Axis\\s+Treasury\\s+Advantage\\s+Fund",
            "Axis Money Market Fund": "Axis\\s+Money\\s+Market\\s+Fund",
            "Axis Corporate Bond Fund": "Axis\\s+Corporate\\s+Bond\\s+Fund",
            "Axis Banking & PSU Debt Fund": "Axis\\s+Banking\\s*(?:&|and)?\\s*PSU\\s+Debt\\s+Fund",
            "Axis Short Duration Fund": "Axis\\s+Short\\s+Duration\\s+Fund",
            "Axis Credit Risk Fund": "Axis\\s+Credit\\s+Risk\\s+Fund",
            "Axis Dynamic Bond Fund": "Axis\\s+Dynamic\\s+Bond\\s+Fund",
            "Axis Strategic Bond Fund": "Axis\\s+Strategic\\s+Bond\\s+Fund",
            "Axis Long Duration Fund": "Axis\\s+Long\\s+Duration\\s+Fund",
            "Axis Gilt Fund": "Axis\\s+Gilt\\s+Fund",
            "Axis Nifty AAA Bond Plus SDL Apr 2026 50:50 ETF": "Axis\\s+Nifty\\s+AAA\\s+Bond\\s+Plus\\s+SDL\\s+Apr\\s+2026\\s+50:?50\\s+ETF",
            "Axis Nifty AAA Bond Plus SDL Apr 2026 50:50 ETF FOF": "Axis\\s+Nifty\\s+AAA\\s+Bond\\s+Plus\\s+S[DO]L\\s+Apr\\s+2026\\s+50:?50\\s+ETF\\s+FOF",
            "Axis US Treasury Dynamic Bond ETF Fund": "Axis\\s+US\\s+Treasury\\s+Dynamic\\s+Bond\\s+ETF\\s+Fund",
            "Axis CRISIL IBX SDL May 2027 Index Fund": "Axis\\s+CRISIL\\s+IBX\\s+SDL\\s+May\\s+2027\\s+Index\\s+Fund",
            "Axis Nifty SDL September 2026 Debt Index Fund": "Axis\\s+Nifty\\s+SDL\\s+September\\s+2026\\s+Debt",
            "Axis CRISIL IBX 50:50 Gilt Plus SDL June 2028 Index Fund": "Axis\\s+CRISIL\\s+IBX.+?Gilt\\s+Plus\\s+SDL\\s+June",
            "Axis CRISIL IBX 50:50 Gilt Plus SDL September 2027 Index Fund": "Axis\\s+CRISIL\\s*IBX.+?Gilt\\s+Plus\\s+SDL\\s+Sep(?:tember)?\\s*2027",
            "Axis CRISIL IBX SDL June 2034 Debt Index Fund": "Axis\\s+CRIS.+?IBX\\s+SDL\\s+June\\s+2034\\s+Debt\\s+Index\\s+Fund",
            "Axis CRISIL-IBX AAA Bond NBFC - Jun 2027 Index Fund": "Axis\\s+CRIS.+?IBX\\s+AAA\\s+Bond\\s+NBFC.+?Jun\\s+2027",
            "Axis CRISIL-IBX AAA Bond Financial Services - Sep 2027 Index Fund": "Axis\\s+CRIS.+?IBX\\s+AAA\\s+Bond\\s+Financial\\s+Services(?:.+?Sep\\s+2027)?",
            "Axis CRISIL-IBX AAA Bond NBFC-HFC - Jun 2027 Index Fund": "Axis\\s+CRIS.+?IBX\\s+AAA\\s+Bond\\s+NBFC.*?HFC",
            "Axis Conservative Hybrid Fund": "Axis\\s+Conservative\\s+Hybrid\\s+Fund",
            "Axis Equity Savings Fund": "Axis\\s+Equity\\s+Savings\\s+Fund",
            "Axis Multi Asset Allocation Fund": "Axis\\s+Multi\\s+Asset\\s+Allocation\\s+Fund",
            "Axis Aggressive Hybrid Fund": "Axis\\s+Aggressive\\s+Hybrid\\s+Fund",
            "Axis Childrens Fund": "Axis\\s+Children\\s*s\\s*Fund",
            "Axis Balanced Advantage Fund": "Axis\\s+Balanced\\s+Advantage\\s+Fund",
            "Axis Arbitrage Fund": "Axis\\s+Arbitrage\\s+Fund",
            "Axis Retirement Fund - Aggressive Plan": "Axis\\s+Retirement\\s+Fund.+?Aggressive\\s+Plan",
            "Axis Retirement Fund - Dynamic Plan": "Axis\\s+Retirement\\s+Fund.+?Dynamic\\s+Plan",
            "Axis Retirement Fund - Conservative Plan": "Axis\\s+Retirement\\s+Fund.+?Conservative\\s+Plan",
            "Axis Gold Fund": "Axis\\s+Gold\\s+Fund",
            "Axis Silver ETF": "Axis\\s+Silver\\s+ETF",
            "Axis Gold ETF": "Axis\\s+Gold\\s+ETF",
            "Axis Silver Fund of Fund": "Axis\\s+Silver\\s+Fund\\s+of\\s+Fund"
        }

In [73]:
axis_list = list(final_dict.keys())
temp = axis_list
for name, regex in axis_regex.items():
    for amc in axis_list:
        if matches:= re.findall(regex,amc, re.IGNORECASE):
            print(matches, regex,name)
            temp.remove(amc)

['axis bluechip fund'] (Axis\s+Large\s+Cap\s+Fund|Axis\s*Bluechip\s*fund) Axis Large Cap Fund
['axis focused fund'] Axis\s+Focused\s+Fund Axis Focused Fund
['axis elss tax saver fund'] Axis\s+ELSS\s+Tax\s+Saver\s+Fund Axis ELSS Tax Saver Fund
['axis growth opportunities fund'] (Axis\s+Large\s*(?:&|and)?\s*Mid\s*Cap\s*Fund|Axis\s*Growth\s*Opportunities\s*Fund) Axis Large & Mid Cap Fund
['axis flexi cap fund'] Axis\s+Flexi\s+Cap\s+Fund Axis Flexi Cap Fund
['axis multicap fund'] Axis\s+Multi\s*cap\s+Fund Axis Multicap Fund
['axis mid cap fund'] Axis\s+Mid\s*cap\s+Fund Axis Midcap Fund
['axis innovation fund'] Axis\s+Innovation\s+Fund Axis Innovation Fund
['axis small cap fund'] Axis\s+Small\s+Cap\s+Fund Axis Small Cap Fund
['axis esg integration strategy fund'] Axis\s+ESG\s+Integration\s+Strategy\s+Fund Axis ESG Integration Strategy Fund
['axis quant fund'] Axis\s+Quant\s+Fund Axis Quant Fund
['axis value fund'] Axis\s+Value\s+Fund Axis Value Fund
['axis business cycles fund'] Axis\s+Busi

In [74]:
temp

['axis income advantage fund of funds',
 'axis children s fund no lock in',
 'axis crisil ibx 70 30 cpse plus sol april',
 'axis nifty aaa bond plus sol apr 2026',
 'axis triple advantage',
 'axis crisil ibx aaa nbfc index jun 2027',
 'axis crisil ibx sdl june 2034 debt index']

In [52]:
with open("final.json","w+") as file:
    json.dump(final_dict,file)

In [ ]:
#NIPPON
# nip_jan = r"C:\Users\Kaustubh.keny\Projects\PDF\Jan 25\Nippon India Mutual Fund\33_31-Jan-25_FS.pdf"
# nip_feb = ""
# nip_mar = r"C:\Users\Kaustubh.keny\Projects\PDF\Mar 25\Nippon India Mutual Fund\33_31-Mar-25_FS.pdf"
# nip_apr = r"C:\Users\Kaustubh.keny\Projects\PDF\Apr 25\Nippon India Mutual Fund\33_30-Apr-25_FS.pdf"
# table_parser = TableParser()
# tables = camelot.read_pdf(nip_apr,flavor="stream",pages="129-140")
# dfs = pd.concat([table.df for table in tables], ignore_index=True)

# sr1 = table_parser._get_matching_row_indices(dfs,["Nippon.+?Fund","Scheme\\s*Name"],thresh=2)
# sr2 = table_parser._get_matching_row_indices(dfs,["minimum application"],thresh=1)
# sr2_expanded = {i for idx in sr2 for i in range(idx, idx + 5)}
# sr1_expanded = {i for idx in sr1 for i in range(idx, idx + 1)}

# all_indices = sr1_expanded | sr2_expanded
# valid_indices = sorted(i for i in all_indices if i in dfs.index)

# filtered_df = dfs.loc[valid_indices].reset_index()
# for idx, rows in filtered_df.iterrows():
#     row_val = " ".join([str(i) for i in rows])
#     row_val = SidKimRegex()._normalize_alphanumeric(row_val)
#     # print(row_val)
#     matches = re.findall(r"Nippon.+?(?:Funds?|ETF|Path|Saver|active|financial|allocation|tunities|duration|psu debt|advantage|small cap 250)\s*(?:of Funds?|Fund of Funds?|Funds?|.+?Plan|FoF)?",row_val, re.IGNORECASE)
#     if matches:
#         print(idx, len(matches))
#         print(matches)

In [ ]:
#BAJAJAJ
# jan = r"C:\Users\Kaustubh.keny\Projects\PDF\Jan 25\Bajaj finserv Mutual Fund\59_31-Jan-25_FS.pdf"
# feb = r"C:\Users\Kaustubh.keny\Projects\PDF\Feb 25\Bajaj finserv Mutual Fund\59_28-Feb-25_FS.pdf"
# mar = r"C:\Users\Kaustubh.keny\Projects\PDF\Mar 25\Bajaj finserv Mutual Fund\59_31-Mar-25_FS.pdf"
# apr = r"C:\Users\Kaustubh.keny\Projects\PDF\Apr 25\Bajaj finserv Mutual Fund\59_30-Apr-25_FS.pdf"


# table_parser = TableParser()
# tables = camelot.read_pdf(apr,flavor="hybrid",pages="11-14")
# dfs = pd.concat([table.df for table in tables], ignore_index=True)
# sc1 = table_parser.get_matching_col_indices(dfs,["Bajaj.+?Fund","SCHEME\\s*NAME"],thresh=20)
# sc2 = table_parser.get_matching_col_indices(dfs,["Jensen","Standard\\s*Deviation","Information\\s*ratio","Portfolio\\s*Quants","Tracking Error","YTM","Average\\s*Maturity","Sharpe"],thresh=10)
# sc3 = table_parser.get_matching_col_indices(dfs,["Average\\s*Maturity","Modified Duration","Macaulay Duration","YTM"], thresh=8)
# all_cols = sorted(set(sc1)) + [sc2[0],sc2[0]+1, sc3[0],sc3[0]+1]
# fdf = dfs.iloc[:, all_cols]
# fdf.columns = ["MUTUAL_FUND"] + [f"METRICS_{i}" for i in range(1, fdf.shape[1])]
# bajaj = re.compile(
#     r"(Bajaj?\\s*finserv.+?)",
#     re.IGNORECASE
# )
# fdf.MUTUAL_FUND = table_parser.clean_series(fdf.MUTUAL_FUND,["normalize_alphanumeric"])
# fdf.MUTUAL_FUND = fdf.MUTUAL_FUND.apply(lambda x: bajaj.findall(x)[0] if isinstance(x, str) and bajaj.findall(x) else "")
# fdf = table_parser.clean_dataframe(fdf,["newline_to_space","str_to_pd_NA"])
# fdf = fdf.dropna(axis=0, how="all").dropna(axis=1, how="all")
# fdf =table_parser.clean_dataframe(fdf,['NA_to_str'])

# data = {}
# for idx, rows in fdf.iterrows():
#     values = list(rows)
#     main_scheme_name = str(values[0]).strip() if not pd.isna(values[0]) else ""
#     if main_scheme_name:
#         temp = main_scheme_name
#         if temp not in data:
#             data[temp] = {"metrics": []}
#         data[temp]["metrics"].append(" ".join(map(str, values[1:])))
    
#     if temp:
#         data[temp]["metrics"].append(" ".join(map(str, values)))


In [ ]:
# #HDFC
# table_parser = TableParser()
# tables = camelot.read_pdf(hdfc1,flavor="lattice",pages="91-93")
# dfs = pd.concat([table.df for table in tables], ignore_index=True)
# sc1 = table_parser._get_matching_col_indices(dfs,["HDFC.+?Fund"],thresh=20)
# sc2 = table_parser._get_matching_col_indices(dfs,["MINIMUM\\s*APPLICATION\\s*AMOUNT","Additional\\s*Purchase"], thresh=20)

# print("Matched columns:", sc1,sc2)
# all_cols = list(set(sc1 + sc2))
# fdf = dfs.iloc[:, all_cols]
# fdf.columns = ["MUTUAL_FUND","MIN_ADD"]
# hdfc_pattern = re.compile(
#     r"(HDFC.+?(?:FUNDS?|ETF|PATH|INDEX|SAVER)\s*(?:OF FUNDS?|FUND OF FUNDS|FOF|.+?PLAN)?)",
#     re.IGNORECASE
# )
# fdf.MUTUAL_FUND = table_parser._clean_series(fdf.MUTUAL_FUND,["normalize_alphanumeric"])
# fdf.MUTUAL_FUND = fdf.MUTUAL_FUND.apply(lambda x: hdfc_pattern.findall(x)[0] if isinstance(x, str) and hdfc_pattern.findall(x) else x)
# fdf = table_parser._clean_dataframe(fdf,["newline_to_space","str_to_pd_NA"])
# fdf = fdf.dropna(axis=0, how="all").dropna(axis=1, how="all")
# fdf =table_parser._clean_dataframe(fdf,['NA_to_str'])

# data = {}
# for idx, rows in fdf.iterrows():
#     values = list(rows)
#     main_scheme_name = values[0]
#     if main_scheme_name not in data:
#         data[main_scheme_name] = {"min_add":values[1]}
#     else:
#         data[main_scheme_name].update({"min_add_one":values[1]})

In [ ]:
# #DSP
# jan = r"C:\Users\Kaustubh.keny\Projects\PDF\Jan 25\DSP Mutual Fund\8_31-Jan-25_FS.pdf"
# feb = r"C:\Users\Kaustubh.keny\Projects\PDF\Feb 25\DSP Mutual Fund\8_28-Feb-25_FS.pdf"
# mar = r"C:\Users\Kaustubh.keny\Projects\PDF\Mar 25\DSP Mutual Fund\8_31-Mar-25_FS.pdf"
# apr = r"C:\Users\Kaustubh.keny\Projects\PDF\Apr 25\DSP Mutual Fund\8_30-Apr-25_FS.pdf"

# table_parser = TableParser()
# tables = camelot.read_pdf(apr,flavor="lattice",pages="107-123")
# dfs = pd.concat([table.df for table in tables], ignore_index=True)
# sc1 = table_parser._get_matching_col_indices(dfs,["DSP.+?Fund"],thresh=12)
# sc2 = table_parser._get_matching_col_indices(dfs,["REGULAR\\s+PLAN","DIRECT\\s+PLAN"], thresh=12)
# sc3 = table_parser._get_matching_col_indices(dfs,["Managing this scheme","total work experience"],thresh=12)
# print("Matched columns:", sc1,sc2,sc3)
# all_cols = list(set(sc1 + sc2 + sc3))
# fdf = dfs.iloc[:, all_cols]
# fdf["LOAD_STRUCTURE"] = fdf.iloc[:, -1]
# fdf.columns = ["MUTUAL_FUND","FUND_MANAGER","MIN_ADD","LOAD_STRUCTURE"]

# dsp_pattern = re.compile(
#     r"(DSP.+?(?:FUNDS?|ETF|PATH|INDEX|SAVER)\s*(?:OF FUNDS?|FUNDs?|FUND OF FUNDS?|FOF|.+?PLAN)?)",
#     re.IGNORECASE
# )

# fdf.MUTUAL_FUND = table_parser._clean_series(fdf.MUTUAL_FUND,["normalize_alphanumeric"])
# fdf.MUTUAL_FUND = fdf.MUTUAL_FUND.apply(lambda x: dsp_pattern.findall(x)[0] if isinstance(x, str) and dsp_pattern.findall(x) else pd.NA)
# fdf = table_parser._clean_dataframe(fdf,["newline_to_space","str_to_pd_NA"])
# fdf = fdf.dropna(axis=0, how="all").dropna(axis=1, how="all")
# fdf =table_parser._clean_dataframe(fdf,['NA_to_str'])

# data = {}
# for idx, rows in fdf.iterrows():
#     values = list(rows)
#     main_scheme_name = values[0]
#     if main_scheme_name not in data:
#         data[main_scheme_name] = {"fund_manager":values[1],"load_structure":values[3],"min_add":values[2]}
#     else:
#          data[main_scheme_name].update({"fund_manager_one":values[1],"load_structure_one":values[3],"min_add_one":values[2]})
        


In [ ]:
def get_proper_fund_names(path: str):
    pattern = "(AXIS.*?(?:FUNDS?|ETF[Ss]?|INDEX|PLAN|PATH|F[Oo]F)\\s*(?:OF\\s*FUNDS?|F[Oo]F|FUNDS?|FUND OF FUNDS?|.+?PLAN)?)"
    title = {} 
    with fitz.open(path) as doc:
        for pgn, page in enumerate(doc):
            text = " ".join(page.get_text("text", clip=(0, 0, 620, 36)).split("\n"))
            text = re.sub("[^A-Za-z0-9\\s\\.,\\-\\(\\)\\+\\%\\:]+", "", text).strip()
            print(text)
            if matches := re.findall(pattern, text, re.DOTALL):
                title[pgn] = " ".join([_ for _ in matches[0].strip().split(" ") if _])
                print(pgn,matches[0])
    return title
path = r"C:\Users\Kaustubh.keny\Projects\PDF\May 25\Axis Mutual Fund\1_31-May-25_FS.pdf"
title = get_proper_fund_names(path)






HOW TO READ A FACTSHEET

AXIS LARGE CAP FUND FACTSHEET (Formerly known as Axis Bluechip Fund)
7 AXIS LARGE CAP FUND 
AXIS FOCUSED FUND FACTSHEET
8 AXIS FOCUSED FUND 
AXIS ELSS TAX SAVER FUND FACTSHEET
9 AXIS ELSS TAX SAVER FUND 
AXIS LARGE  MID CAP FUND FACTSHEET (Formerly known as Axis Growth Opportunities Fund)
10 AXIS LARGE  MID CAP FUND 
AXIS FLEXI CAP FUND FACTSHEET
11 AXIS FLEXI CAP FUND 
AXIS MULTICAP FUND FACTSHEET
12 AXIS MULTICAP FUND 
AXIS MIDCAP FUND FACTSHEET
13 AXIS MIDCAP FUND 
AXIS INNOVATION FUND FACTSHEET
14 AXIS INNOVATION FUND 
AXIS SMALL CAP FUND FACTSHEET
15 AXIS SMALL CAP FUND 
AXIS ESG INTEGRATION STRATEGY FUND FACTSHEET
16 AXIS ESG INTEGRATION STRATEGY FUND 
AXIS QUANT FUND FACTSHEET
17 AXIS QUANT FUND 
AXIS VALUE FUND FACTSHEET
18 AXIS VALUE FUND 
AXIS BUSINESS CYCLES FUND FACTSHEET
19 AXIS BUSINESS CYCLES FUND 
AXIS INDIA MANUFACTURING FUND FACTSHEET
20 AXIS INDIA MANUFACTURING FUND 
AXIS CONSUMPTION FUND FACTSHEET
21 AXIS CONSUMPTION FUND 
AXIS MOMENTUM

In [2]:
def _normalize_alphanumeric(text: str) -> str:
    if not isinstance(text,str):
        return text
    text = re.sub(r"[^a-zA-Z0-9]+", " ", str(text))
    return re.sub(r"\s+", " ", text).strip().lower()

def extract_clipped_data(input:str, pages:list, bboxes:list):
        
        document = fitz.open(input)
        final_list = []
    
        for pgn in pages:
            page = document[pgn]
            
            all_blocks = [] #store every data from bboxes
            
            for bbox in bboxes:
                blocks, seen_blocks = [], set()  #store unique blocks based on content and bbox
                
                page_blocks = page.get_text('dict', clip=bbox)['blocks']
                for block in page_blocks:
                    if block['type'] == 0 and 'lines' in block: #type 0 means text block
                        #hash_key
                        block_key = (tuple(block['bbox']), tuple(tuple(line['spans'][0]['text'] for line in block['lines'])))
                        if block_key not in seen_blocks:
                            seen_blocks.add(block_key)
                            blocks.append(block)

                sorted_blocks = sorted(blocks, key=lambda x: (x['bbox'][1], x['bbox'][0]))
                all_blocks.append(sorted_blocks)

            final_list.append({
                "pgn": pgn,
                "block": all_blocks #will be list[list,list,..]
            })

        document.close()
        return final_list
    
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        sorted_blocks = sorted(blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in sorted_blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

      
        final_list.append({
            "pgn": pgn,
            "blocks": extracted_blocks
        })

    doc.close()

    return final_list
  
def get_clipped_data(input:str, bboxes:list[set], *args):
    
        document = fitz.open(input)
        final_list = []
        if args:
            pages = list(args)
        else:
            pages = [i for i in document.page_count]
        
        for pgn in pages:
            page = document[pgn]

            blocks = []
            for bbox in bboxes:
                blocks.extend(page.get_text('dict', clip = bbox)['blocks']) #get all blocks
            
            filtered_blocks = [block for block in blocks if block['type']== 0 and 'lines' in block]
            # sorted_blocks = sorted(filtered_blocks, key= lambda x: (x['bbox'][1], x['bbox'][0]))
            
             # Extract text from sorted blocks
            extracted_text = []
            for block in filtered_blocks:
                block_text = []
                for line in block['lines']:
                    line_text = " ".join(span['text'] for span in line['spans'])
                    block_text.append(line_text)
                extracted_text.append("\n".join(block_text))
            
            final_list.append({
            "pgn": pgn,
            "block": filtered_blocks,
            "text": extracted_text
            })
            
            
        document.close()
        return final_list
    
def extract_clipped_text_all_pages(pdf_path, clip_coords):
    results = {}
    doc = fitz.open(pdf_path)
    clip_rect = fitz.Rect(*clip_coords)
    try:
        for page_number, page in enumerate(doc):
            text = page.get_text("text", clip=clip_rect).strip()
            results[page_number] = text
    finally:
        doc.close()
    return results

In [3]:
data = Helper.get_all_pdf_data(mutual_fund["Quant Mutual Fund"])